# Imports

Before starting the import into Spotify you need to go to Spotify Developers and make a new App. From there onwards you need to follow this Notebook.

In [14]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
# Make a file in your directory containing username, client_id and client_secret
import keys

In [15]:
song_ids = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/df_audio_features_5000_cleaned_whitespaces.csv')

In [16]:
song_ids.drop(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'duration_ms', 'time_signature'], inplace=True)

In [17]:
songss = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists.csv')

In [18]:
songs = songss.merge(song_ids, on=['name', 'artist'], how='left')

In [19]:
scope='playlist-modify-public'
username= keys.username
redirectUri = 'http://localhost:8080/callback'
client_id=keys.client_id
client_secret=keys.client_secret
playlist_name = 'My Once Playlist'
playlist_description = 'App for my bootcamp project Moosaic. Create playlists through clustering.'

In [20]:
token = SpotifyOAuth(scope=scope,username=username,client_id=client_id,
                     client_secret=client_secret, redirect_uri=redirectUri)

In [21]:
spotifyObject = spotipy.Spotify(auth_manager=token)

In [22]:
# # Create Playlists
# playlist_df = pd.DataFrame(columns=['playlist_name', 'cluster', 'id'])

# for clust in songs['cluster'].unique():
#     playlist_name = f'Playlist Cluster {clust}'
#     playlist_description = 'App for my bootcamp project Moosaic. Create playlists through clustering.'
    
#     try:
#         playlist_response = spotifyObject.user_playlist_create(user=username, name=playlist_name, 
#                                                          description=playlist_description, public=True)
#         playlist_id = playlist_response['id']
        
#         new_row = pd.DataFrame({'playlist_name': [playlist_name], 'cluster': [clust], 'id': [playlist_id]})

#         playlist_df = pd.concat([playlist_df, new_row], ignore_index=True)
        
#         print(f"Playlist created with ID: {playlist_id}")
#     except Exception as e:
#         print(f"An error occurred: {e}")

Playlist created with ID: 7JiQ59rGvx33qNcb0UuBrW
Playlist created with ID: 3WS4UR8P93IZO9gZ3WwhUx
Playlist created with ID: 3dM2ePptXCxGvpY0q9mqNy
Playlist created with ID: 5WfHHokqJni2C5RWDT6Wj3
Playlist created with ID: 3ekpBMyWH8HXqFPpl2wj2a
Playlist created with ID: 0Qz6WKRtleyVXjpOMrvmc4
Playlist created with ID: 1N7yFTYrJ3r7GaTblnSDgK
Playlist created with ID: 5TUt4ahrT55jAANp7PJiYZ
Playlist created with ID: 4NwrQxU1T8q3uhdCmWe7ry
Playlist created with ID: 7KC3xi1piQcfiqn7f2vYxA
Playlist created with ID: 4VuDkBDrwt2XuSTfDLU2ws
Playlist created with ID: 5Gv1hJKkEBS0yrI1f99riB
Playlist created with ID: 5yx0o7Ghy44zV6J51Nk6Lq
Playlist created with ID: 2qVicSaGkPu6LuVe4dlUlg
Playlist created with ID: 081DcWfAjXVqDRaqZVeJUI
Playlist created with ID: 7zwr5Vsd3nNkjWuC4nTznN
Playlist created with ID: 65w3PdtnaZnqqbaixD7dq5
Playlist created with ID: 0l4Rff91OekPA2aUvOcp45
Playlist created with ID: 01R5zEhfSSugc93LLoGJs6
Playlist created with ID: 5cevhgMHmIjumtzZu6wjxm
Playlist created wit

In [23]:
playlists = playlist_df.copy()

In [24]:
playlists['id'] = playlists['id'].str.strip()

In [25]:
playlists.to_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists_ids_clusters', index=False)

In [26]:
playlists = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists_ids_clusters')

In [27]:
playlists.head()

,playlist_name,cluster,id
0,Playlist Cluster 24,24,7JiQ59rGvx33qNcb0UuBrW
1,Playlist Cluster 47,47,3WS4UR8P93IZO9gZ3WwhUx
2,Playlist Cluster 17,17,3dM2ePptXCxGvpY0q9mqNy
3,Playlist Cluster 21,21,5WfHHokqJni2C5RWDT6Wj3
4,Playlist Cluster 3,3,3ekpBMyWH8HXqFPpl2wj2a


In [28]:
def divide_into_chunks(lst, n):
    """Divide a list into n roughly equal parts."""
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [32]:
songs.head()

,name,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster,id,html
0,Se Eu Quiser Falar Com Deus ...,Gilberto Gil,0.680455,0.2590,0.763897,0.076797,0.696787,0.000060,0.987842,0.310660,0.515800,24,1n7JnwviZ7zf0LR1tcGFq7,https://open.spotify.com/track/1n7JnwviZ7zf0LR...
1,Saudade De Bahia ...,Antônio Carlos Jobim,0.767322,0.3990,0.771967,0.037691,0.217871,0.000002,0.108409,0.703553,0.584322,47,5QGM1U0eCYrQuwSJwTm5Zq,https://open.spotify.com/track/5QGM1U0eCYrQuwS...
2,"Canta Canta, Minha Gente ...",Martinho Da Vila,0.880041,0.7300,0.798018,0.377996,0.454819,0.000064,0.125633,0.918782,0.437862,17,0NLIFSZxPzQhCwnkn5PJYs,https://open.spotify.com/track/0NLIFSZxPzQhCwn...
3,"Canta Canta, Minha Gente ...",Martinho Da Vila,0.880041,0.7300,0.798018,0.377996,0.454819,0.000064,0.125633,0.918782,0.437862,17,57E1ka54d8zrIy0VzE1EAV,https://open.spotify.com/track/57E1ka54d8zrIy0...
4,Mulher Eu Sei ...,Chico César,0.729059,0.0502,0.682811,0.051307,0.882530,0.000041,0.391084,0.531980,0.499098,21,3mXqOdlLE1k67WsAxryPFs,https://open.spotify.com/track/3mXqOdlLE1k67Ws...


In [33]:
for clust in songs['cluster'].unique():
    cluster_songs = songs[songs['cluster'] == clust]
    # Convert song IDs into the desired format
    list_of_songs = [f'{row["html"]}' for _, row in cluster_songs.iterrows()]
    print(list_of_songs)
    playlist_id = playlists[playlists['cluster'] == clust]['id'].iloc[0]
    print(f'Playlist id is : {playlist_id} and the cluster is {clust}')
    
    try:
        spotifyObject.playlist_add_items(playlist_id=playlist_id, items=list_of_songs, position=None)
        print(f"Added {len(list_of_songs)} tracks to playlist {playlist_id}")
    except Exception as e:
        print(f"Error adding tracks to playlist: {e}")
        print(f"Failed chunk: {list_of_songs}")

['https://open.spotify.com/track/1n7JnwviZ7zf0LR1tcGFq7', 'https://open.spotify.com/track/37H0kOistSqxRH8rObQeIe', 'https://open.spotify.com/track/2hjDxNmryN9ziHAIYiRksM', 'https://open.spotify.com/track/6T0uy8Np5fDNpb8uYA5kXg', 'https://open.spotify.com/track/794003xwvF9cGLRSY0dlY7', 'https://open.spotify.com/track/2Baiob9mbEkOJZxD6Yro3k', 'https://open.spotify.com/track/2ctBnKIGC5KoWUg8reeCTL', 'https://open.spotify.com/track/4Q3nQek3cWJqlRVPrvsbdJ', 'https://open.spotify.com/track/38hizYSleQ0861n2jyS7MR', 'https://open.spotify.com/track/7juNpUD2eKlSPBTdgr8EGC', 'https://open.spotify.com/track/6QG28FulnubhxvNnNS3D8S', 'https://open.spotify.com/track/3NNRdQduyl3MzJQE1chyRb', 'https://open.spotify.com/track/307XEC1IUwUs9ojlEFwH7f', 'https://open.spotify.com/track/2fXNIyzcOjc5Otknzsfi2O', 'https://open.spotify.com/track/3jQa7pofP3CHRTt4Xnoyt6', 'https://open.spotify.com/track/51Fi8flrG3IHrFeU4G198k', 'https://open.spotify.com/track/7iPfY3lXOMH77BoZstQUhc', 'https://open.spotify.com/trac

HTTP Error for POST to https://api.spotify.com/v1/playlists/3WS4UR8P93IZO9gZ3WwhUx/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.
HTTP Error for POST to https://api.spotify.com/v1/playlists/3dM2ePptXCxGvpY0q9mqNy/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.


Added 25 tracks to playlist 7JiQ59rGvx33qNcb0UuBrW
['https://open.spotify.com/track/5QGM1U0eCYrQuwSJwTm5Zq', 'https://open.spotify.com/track/5BUYtDh62uirLH4vMnE2dA', 'https://open.spotify.com/track/5p62Lc3ALej4hOuClxLGUx', 'https://open.spotify.com/track/5dyxXwO5tacUf6PazUC2y7', 'https://open.spotify.com/track/1DFFUsN49AUonUcTiQNnjI', 'https://open.spotify.com/track/0HdeVf8f8wVkrYNYbF0u2t', 'https://open.spotify.com/track/7aghcqWLL9hGMIb0VFdbyD', 'https://open.spotify.com/track/6scUEism6pBnY0IXYr8fw3', 'https://open.spotify.com/track/0dRh2oB1UmTeUKfyOLT3hf', 'https://open.spotify.com/track/5DsIC4wlfuBxVBXMZmfTSk', 'https://open.spotify.com/track/2os0aK782bakCPmjow0SU0', 'https://open.spotify.com/track/7DDU2GljfW6HV5SQ7ZNnmE', 'https://open.spotify.com/track/5FLPz4PIuy1dehS0JMuEFH', 'https://open.spotify.com/track/3zOILDgiueK2pFLXbxPPAJ', 'https://open.spotify.com/track/7CHi4DtfK4heMlQaudCuHK', 'https://open.spotify.com/track/4QbX1M5JblYzGTVwdZTgfb', 'https://open.spotify.com/track/1WSG

HTTP Error for POST to https://api.spotify.com/v1/playlists/5WfHHokqJni2C5RWDT6Wj3/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.
HTTP Error for POST to https://api.spotify.com/v1/playlists/3ekpBMyWH8HXqFPpl2wj2a/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.


Error adding tracks to playlist: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/5WfHHokqJni2C5RWDT6Wj3/tracks:
 You can add a maximum of 100 tracks per request., reason: None
Failed chunk: ['https://open.spotify.com/track/3mXqOdlLE1k67WsAxryPFs', 'https://open.spotify.com/track/33u3dO0JWjerYpFN61J3ug', 'https://open.spotify.com/track/65GsUbgQju5Mh0Ab7CRUWB', 'https://open.spotify.com/track/2rIzPAaLY6ocCanj5H7edQ', 'https://open.spotify.com/track/0gwNAMVAXRvERlC16eZXyA', 'https://open.spotify.com/track/4WfT503LMc9zLewdsok9un', 'https://open.spotify.com/track/1htDFdG7K6pW07cuIanjGG', 'https://open.spotify.com/track/7bM4h5zu7FkChGbSeozZVN', 'https://open.spotify.com/track/22QBm5IGpx49Ggg7b0DMhL', 'https://open.spotify.com/track/1b0Mc4h1XFArQAnQYBmIQb', 'https://open.spotify.com/track/0OIXsw1P2SZvF6T2qsFFzS', 'https://open.spotify.com/track/4Wzym5Rbnn8Ny8rFqEObm1', 'https://open.spotify.com/track/0KY71RFPZJPqR04jOQFUw2', 'https://open.spotify.com/track/1dtPIHAk2W9tn9WVW6h

...

Error adding tracks to playlist: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/1HK0MZAzywDEVr3ZPBi0NI/tracks:
 You can add a maximum of 100 tracks per request., reason: None
Failed chunk: ['https://open.spotify.com/track/3XpXhVtZwqh2eM5d9ieXT5', 'https://open.spotify.com/track/1aIZ1YDVQK9nTTwxVnD8ah', 'https://open.spotify.com/track/4c6vZqYHFur11FbWATIJ9P', 'https://open.spotify.com/track/4fiOTntQKr24p07FvQDHZE', 'https://open.spotify.com/track/4PWxt9Fy949kUtkEi2GI5V', 'https://open.spotify.com/track/4oiwzl01mQDqRWFIa7VI3d', 'https://open.spotify.com/track/72ys4n6BKV83iWdPa51Dk2', 'https://open.spotify.com/track/5oAgI7Pzg6vxuKrfapg3Pi', 'https://open.spotify.com/track/15ccEhr9NN3Blmdr0hQwj9', 'https://open.spotify.com/track/3VGCxo3ojR9PJQ81IqCE3p', 'https://open.spotify.com/track/5ABDsQEkVZ1ECGPGQl6Yfr', 'https://open.spotify.com/track/2l0h4aBFLp9HdoaNdCTlbW', 'https://open.spotify.com/track/7jok5zvxnrQuK2N7psZHyN', 'https://open.spotify.com/track/233N4WwgPBme5byY337

HTTP Error for POST to https://api.spotify.com/v1/playlists/4Iv7PazDx0FNtPilutfS0K/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.
HTTP Error for POST to https://api.spotify.com/v1/playlists/19NH3gLS9CNVm9itItCucE/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.


Error adding tracks to playlist: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/4Iv7PazDx0FNtPilutfS0K/tracks:
 You can add a maximum of 100 tracks per request., reason: None
Failed chunk: ['https://open.spotify.com/track/0w9bDd1FaBpYFXlILpWe3P', 'https://open.spotify.com/track/2bZYJvVRN5EqIvAUFhUNls', 'https://open.spotify.com/track/6Nd6ntkzr4t8o1FKPGOSMt', 'https://open.spotify.com/track/3CBoWhq9cd7WGQZ9EDQVnn', 'https://open.spotify.com/track/0L7zm6afBEtrNKo6C6Gj08', 'https://open.spotify.com/track/15z9QEJOBV6hDdWsdZwDKP', 'https://open.spotify.com/track/5F6ZrWLngDICyjazxM1gNq', 'https://open.spotify.com/track/0dpFb4qv92TE9R00Lq4SAW', 'https://open.spotify.com/track/7t5riom6kdqXkb1VzTlAaH', 'https://open.spotify.com/track/3sSonVXqDeoEFj2lM7mpYT', 'https://open.spotify.com/track/5XnyWvKPVgJsVKmUjFbMv3', 'https://open.spotify.com/track/45QRBDrfLXVvJn2VeGPQCn', 'https://open.spotify.com/track/6QzjXmQ1mHNYHrwNurA9vJ', 'https://open.spotify.com/track/1erRDf98WX0VllciGPY

HTTP Error for POST to https://api.spotify.com/v1/playlists/3e7v3Jzak34OB4JQSQwpeL/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.
HTTP Error for POST to https://api.spotify.com/v1/playlists/6jAMoHffGOjJfBq5mkjlRe/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.


Error adding tracks to playlist: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3e7v3Jzak34OB4JQSQwpeL/tracks:
 You can add a maximum of 100 tracks per request., reason: None
Failed chunk: ['https://open.spotify.com/track/4tQoPPpXKILHrBO9w3dnLr', 'https://open.spotify.com/track/6IcniICSO1Q72jmxvLIy6j', 'https://open.spotify.com/track/0Ck1w9drSHl9NrWV9diIXv', 'https://open.spotify.com/track/18JxYMTSFKZlxqdNFrOs1r', 'https://open.spotify.com/track/0icJXGequFzOB37jNOLaeU', 'https://open.spotify.com/track/2kyEgPaAW8wdpvevPnkf0Z', 'https://open.spotify.com/track/7a1HnIeNICJhTP0GqGS0mU', 'https://open.spotify.com/track/1XPUfZx0Nhl7Tl1AT2LKos', 'https://open.spotify.com/track/0NJdXrN8zI2PHPzMuflFwU', 'https://open.spotify.com/track/6wgBdqFfalobMk33lbND2s', 'https://open.spotify.com/track/4r6dN7hJF960vCPFyvxQ4l', 'https://open.spotify.com/track/5RDTMKXIXn3qP0V2G5zeCM', 'https://open.spotify.com/track/1Hnp1eblIw5zvF4BcmcXl0', 'https://open.spotify.com/track/1gZ73kjVHsICCNLKRUX

In [34]:
clusters = [43, 0, 16, 31, 2, 36, 4, 27, 30, 41, 10, 14, 13, 20, 6, 9, 45, 1, 33, 8, 18, 26, 3, 21, 17, 47]
# change cluster to songs['cluster'].unique()
for clust in clusters:
    cluster_songs = songs[songs['cluster'] == clust]
    # Convert song IDs into the desired format
    list_of_songs = [f'{row["html"]}' for _, row in cluster_songs.iterrows()]
    print(list_of_songs)
    playlist_id = playlists[playlists['cluster'] == clust]['id'].iloc[0]
    print(f'Playlist id is : {playlist_id}')
    chunks = divide_into_chunks(list_of_songs, 4)

    for chunk in chunks:
        try:
            spotifyObject.playlist_add_items(playlist_id=playlist_id, items=chunks, position=None)
            print(f"Added {len(chunk)} tracks to playlist {playlist_id}")
        except Exception as e:
            print(f"Error adding tracks to playlist: {e}")
            print(f"Failed chunk: {chunk}")

['https://open.spotify.com/track/3568o6514xWX3KfA7tUYPY', 'https://open.spotify.com/track/3dfO5e6kvudYYQwWiLbBoo', 'https://open.spotify.com/track/774kFk9YCPyTYR7sSIDqum', 'https://open.spotify.com/track/1XM1xWWhp0PvgAb9PthNPd', 'https://open.spotify.com/track/0U6W8pajvbJhGSoCFnggWJ', 'https://open.spotify.com/track/3EjNZuQqmUUuEEM1rXBxcn', 'https://open.spotify.com/track/3ef3N03PUANmE4zPUVWF3D', 'https://open.spotify.com/track/3cmEh49U5WRfPcFipIu7LY', 'https://open.spotify.com/track/2CvtsZt7wGW1iIFniV5CAw', 'https://open.spotify.com/track/66opfMXBdeH53Ew5X6XydK', 'https://open.spotify.com/track/0RbaR3BU0zI8ihE71unIFg', 'https://open.spotify.com/track/0pgOBcrcBjvgTVQVyW2azU', 'https://open.spotify.com/track/0l4jYrGwJti1vxNtDdP9FP', 'https://open.spotify.com/track/18f8TAm7J8QW0mM3j6CN74', 'https://open.spotify.com/track/1Q2cltswPY0iojjk0YtN5n', 'https://open.spotify.com/track/1gSdmvrU9uR5pecu7BUwoM', 'https://open.spotify.com/track/5tUxhJPbbOB4vEwm7ax1m2', 'https://open.spotify.com/trac

In [ ]:
playlist_id

In [ ]:
list_of_songs

In [ ]:
spotifyObject.user_playlist_add_tracks(user=username,playlist_id=playlist_id['id'],tracks=list_of_songs)